In [1]:
%pylab inline
import pandas as pd
from sklearn import cross_validation as cv

Populating the interactive namespace from numpy and matplotlib


/home/dylan/GitHub/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv("data/train-santander.csv")
X_orig = data.drop(['TARGET','ID'], axis=1)
y_orig = data['TARGET']
X_train, X_test, \
    y_train, y_test = cv.train_test_split(X_orig, y_orig, 
                                          test_size=0.33, stratify=y_orig)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.as_matrix()
y_test = y_test.as_matrix()

In [19]:
# validation set for early stopping
X_train, X_val, \
    y_train, y_val = cv.train_test_split(X_train, y_train, 
                                          test_size=0.10, stratify=y_train)

In [15]:
# weights for training/val set
from collections import Counter

sample_weights = np.ones(len(y_train))
sample_weights
c = Counter(y_train)
positive_weight = c[0]/float(c[1])
sample_weights[y_train == 1] = positive_weight

In [5]:
help(xgb.XGBClassifier())

Help on XGBClassifier in module xgboost.sklearn object:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |      Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of boosted trees to fit.
 |  silent : boolean
 |      Whether to print messages while running boosting.
 |  objective : string or callable
 |      Specify the learning task and the corresponding learning objective or
 |      a custom objective function to be used (see note below).
 |  nthread : int
 |      Number of parallel threads used to run xgboost.
 |  gamma : float
 |      Minimum loss reduction required to make a further partition on a leaf node of the tree.
 |  min_child_weight : int
 |      Minimum sum of instance weight(hessian) needed in a child.
 |  max_delta_step : int
 |      M

In [3]:
from sklearn import grid_search
from sklearn import cross_validation as cv
from sklearn import ensemble, pipeline
from sklearn import feature_selection as select
from sklearn import preprocessing as prep
import xgboost as xgb
from time import time
pipe = pipeline.Pipeline([
    ('remover', select.VarianceThreshold()),
    ('scaler', prep.StandardScaler()),
])

/home/dylan/GitHub/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [49]:
# preprocessing train and val set
X_train_preproc = pipe.fit_transform(X_train)
X_val_preproc = pipe.transform(X_val)

In [71]:
clf = xgb.XGBClassifier(silent=False, objective="binary:logistic")
clf

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [55]:
# I CAN'T BELIEVE THE SPEED AND ACCURACY!
clf.fit(X_train_preproc, y_train, sample_weight=sample_weights, eval_set=[(X_val_preproc, y_val)], eval_metric='auc',
       early_stopping_rounds=10, verbose=True)

Will train until validation_0 error hasn't decreased in 10 rounds.
[0]	validation_0-auc:0.761118
[1]	validation_0-auc:0.789964
[2]	validation_0-auc:0.791681
[3]	validation_0-auc:0.790525
[4]	validation_0-auc:0.790725
[5]	validation_0-auc:0.790679
[6]	validation_0-auc:0.790482
[7]	validation_0-auc:0.790537
[8]	validation_0-auc:0.790273
[9]	validation_0-auc:0.789429
[10]	validation_0-auc:0.800502
[11]	validation_0-auc:0.800509
[12]	validation_0-auc:0.800436
[13]	validation_0-auc:0.803931
[14]	validation_0-auc:0.804059
[15]	validation_0-auc:0.813096
[16]	validation_0-auc:0.816655
[17]	validation_0-auc:0.819271
[18]	validation_0-auc:0.821834
[19]	validation_0-auc:0.820263
[20]	validation_0-auc:0.820669
[21]	validation_0-auc:0.820606
[22]	validation_0-auc:0.821507
[23]	validation_0-auc:0.821099
[24]	validation_0-auc:0.820330
[25]	validation_0-auc:0.822198
[26]	validation_0-auc:0.822535
[27]	validation_0-auc:0.821500
[28]	validation_0-auc:0.822712
[29]	validation_0-auc:0.822729
[30]	validati

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [72]:
seed=np.random.randint(0,10000)
clf.set_params(**{"seed" : seed})

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1192, silent=False, subsample=1)

In [74]:
parameters = {"n_estimators" : [100,300,500], "gamma" : [0,1,10], "max_depth" : [6, 12, 18], 
              "reg_lambda" : [1, 3, 5]}

In [75]:
# optimizing hyperparameters using GridSearchCV
gs = grid_search.GridSearchCV(clf, 
    parameters, scoring='roc_auc', verbose=2, n_jobs=1, cv=3, 
    fit_params= {"sample_weight" : sample_weights, "eval_set": [(X_val_preproc, y_val)], "eval_metric":'auc',
                "early_stopping_rounds" : 10, "verbose" : True})

In [76]:
gs.fit(X_train_preproc, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] n_estimators=100, reg_lambda=1, max_depth=6, gamma=0 ............
Will train until validation_0 error hasn't decreased in 10 rounds.
[0]	validation_0-auc:0.801176
[1]	validation_0-auc:0.801652
[2]	validation_0-auc:0.807594
[3]	validation_0-auc:0.808639
[4]	validation_0-auc:0.812368
[5]	validation_0-auc:0.814081
[6]	validation_0-auc:0.815980
[7]	validation_0-auc:0.817718
[8]	validation_0-auc:0.819182
[9]	validation_0-auc:0.819328
[10]	validation_0-auc:0.818473
[11]	validation_0-auc:0.820189
[12]	validation_0-auc:0.820263
[13]	validation_0-auc:0.819983
[14]	validation_0-auc:0.819095
[15]	validation_0-auc:0.820137
[16]	validation_0-auc:0.820673
[17]	validation_0-auc:0.820180
[18]	validation_0-auc:0.819846
[19]	validation_0-auc:0.819576
[20]	validation_0-auc:0.820179
[21]	validation_0-auc:0.819960
[22]	validation_0-auc:0.820037
[23]	validation_0-auc:0.823596
[24]	validation_0-auc:0.824698
[25]	validation_0-auc:0.825232
[26]

[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed: 18.1min
[Parallel(n_jobs=1)]: Done 161 tasks       | elapsed: 90.8min


[CV] .. n_estimators=500, reg_lambda=5, max_depth=18, gamma=1 -  41.9s
[CV] n_estimators=500, reg_lambda=5, max_depth=18, gamma=1 ...........
Will train until validation_0 error hasn't decreased in 10 rounds.
[0]	validation_0-auc:0.780835
[1]	validation_0-auc:0.782866
[2]	validation_0-auc:0.783027
[3]	validation_0-auc:0.811083
[4]	validation_0-auc:0.816303
[5]	validation_0-auc:0.821582
[6]	validation_0-auc:0.825841
[7]	validation_0-auc:0.823634
[8]	validation_0-auc:0.824171
[9]	validation_0-auc:0.822618
[10]	validation_0-auc:0.822095
[11]	validation_0-auc:0.821957
[12]	validation_0-auc:0.823115
[13]	validation_0-auc:0.824041
[14]	validation_0-auc:0.823203
[15]	validation_0-auc:0.821671
[16]	validation_0-auc:0.820174
Stopping. Best iteration:
[6]	validation_0-auc:0.825841

[CV] .. n_estimators=500, reg_lambda=5, max_depth=18, gamma=1 -  13.9s
[CV] n_estimators=100, reg_lambda=1, max_depth=6, gamma=10 ...........
Will train until validation_0 error hasn't decreased in 10 rounds.
[0]	vali

[Parallel(n_jobs=1)]: Done 243 out of 243 | elapsed: 122.5min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1192, silent=False, subsample=1),
       fit_params={'eval_set': [(array([[ 0.04019, -0.474  , ..., -0.01981, -0.32689],
       [ 0.04019, -0.16349, ..., -0.01981,  2.23548],
       ...,
       [ 0.04019, -0.474  , ..., -0.01981,  0.00303],
       [ 0.04019, -0.00823, ..., -0.01981,  0.40912]]), array([0, 0, ..., 0, 0]))], 'sample_weight': array([ 1.,  1., ...,  1.,  1.]), 'early_stopping_rounds': 10, 'eval_metric': 'auc', 'verbose': True},
       iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 300, 500], 'reg_lambda': [1, 3, 5], 'max_depth': [6, 12, 18], 'gamma': [0, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True,

In [78]:
# but this doesn't use "best" iteration
print gs.best_params_
print gs.best_score_

{'n_estimators': 100, 'reg_lambda': 1, 'max_depth': 18, 'gamma': 10}
0.826959348437


In [97]:
y_train_weights = np.ones(len(y_train))
y_train_weights[y_train == 1] = positive_weight

In [99]:
clf = xgb.XGBClassifier(silent=False, objective="binary:logistic", n_estimators=300, reg_lambda=1, max_depth=6)
clf.fit(X_train_preproc, y_train, sample_weight=y_train_weights, eval_set=[(X_val_preproc, y_val)], eval_metric='auc',
       early_stopping_rounds=20, verbose=True)


Will train until validation_0 error hasn't decreased in 20 rounds.
[0]	validation_0-auc:0.818759
[1]	validation_0-auc:0.825053
[2]	validation_0-auc:0.826355
[3]	validation_0-auc:0.828881
[4]	validation_0-auc:0.828952
[5]	validation_0-auc:0.829102
[6]	validation_0-auc:0.831067
[7]	validation_0-auc:0.832058
[8]	validation_0-auc:0.829336
[9]	validation_0-auc:0.829668
[10]	validation_0-auc:0.829987
[11]	validation_0-auc:0.829026
[12]	validation_0-auc:0.827624
[13]	validation_0-auc:0.828716
[14]	validation_0-auc:0.829488
[15]	validation_0-auc:0.829967
[16]	validation_0-auc:0.830265
[17]	validation_0-auc:0.830634
[18]	validation_0-auc:0.830646
[19]	validation_0-auc:0.830832
[20]	validation_0-auc:0.830416
[21]	validation_0-auc:0.831062
[22]	validation_0-auc:0.830597
[23]	validation_0-auc:0.831397
[24]	validation_0-auc:0.831212
[25]	validation_0-auc:0.830958
[26]	validation_0-auc:0.830904
[27]	validation_0-auc:0.830396
Stopping. Best iteration:
[7]	validation_0-auc:0.832058



XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [93]:
y_val_weights = np.ones(len(y_val))
y_val_weights[y_val == 1] = positive_weight

In [100]:
from sklearn import metrics
probs = clf.predict_proba(X_val_preproc)
preds = np.argmax(probs, axis=1)
print("AUC: {}".format(metrics.roc_auc_score(y_val, preds, sample_weight=y_val_weights)))
print("Loss: {}".format(metrics.log_loss(y_val, probs, sample_weight=y_val_weights)))
print("F1 Score: {}".format(metrics.f1_score(y_val, preds, sample_weight=y_val_weights)))
print metrics.classification_report(y_val, preds)

AUC: 0.774336560802
Loss: 0.509352004171
F1 Score: 0.769488159611
             precision    recall  f1-score   support

          0       0.99      0.80      0.88      4892
          1       0.13      0.75      0.23       202

avg / total       0.95      0.79      0.86      5094



In [106]:
clf = xgb.XGBClassifier(silent=False, objective="binary:logistic", n_estimators=300, reg_lambda=3, max_depth=6)
clf.fit(X_train_preproc, y_train, sample_weight=y_train_weights, eval_set=[(X_val_preproc, y_val)], eval_metric='auc',
       early_stopping_rounds=20, verbose=True)
##
probs = clf.predict_proba(X_val_preproc)
preds = np.argmax(probs, axis=1)
print("AUC: {}".format(metrics.roc_auc_score(y_val, preds, sample_weight=y_val_weights)))
print("Loss: {}".format(metrics.log_loss(y_val, probs, sample_weight=y_val_weights)))
print("F1 Score: {}".format(metrics.f1_score(y_val, preds, sample_weight=y_val_weights)))
print metrics.classification_report(y_val, preds)

Will train until validation_0 error hasn't decreased in 20 rounds.
[0]	validation_0-auc:0.821002
[1]	validation_0-auc:0.824452
[2]	validation_0-auc:0.828810
[3]	validation_0-auc:0.832307
[4]	validation_0-auc:0.831914
[5]	validation_0-auc:0.831926
[6]	validation_0-auc:0.833254
[7]	validation_0-auc:0.834188
[8]	validation_0-auc:0.830874
[9]	validation_0-auc:0.831205
[10]	validation_0-auc:0.831465
[11]	validation_0-auc:0.831622
[12]	validation_0-auc:0.831694
[13]	validation_0-auc:0.830648
[14]	validation_0-auc:0.832049
[15]	validation_0-auc:0.832450
[16]	validation_0-auc:0.833413
[17]	validation_0-auc:0.833645
[18]	validation_0-auc:0.834014
[19]	validation_0-auc:0.834700
[20]	validation_0-auc:0.834759
[21]	validation_0-auc:0.834858
[22]	validation_0-auc:0.834707
[23]	validation_0-auc:0.835029
[24]	validation_0-auc:0.834965
[25]	validation_0-auc:0.834335
[26]	validation_0-auc:0.834498
[27]	validation_0-auc:0.834482
[28]	validation_0-auc:0.834657
[29]	validation_0-auc:0.835191
[30]	validati

In [5]:
from collections import Counter
X_orig_preproc = pipe.fit_transform(X_orig)
# weights for original set
c = Counter(y_orig)
positive_weight = c[0]/float(c[1])
y_orig_weights = np.ones(len(y_orig))
y_orig_weights[y_orig == 1] = positive_weight

clf = xgb.XGBClassifier(silent=False, objective="binary:logistic", n_estimators=300, reg_lambda=3, max_depth=6)
clf.fit(X_orig_preproc, y_orig, sample_weight=y_orig_weights)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=3,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [6]:
test_data = pd.read_csv("data/test-santander.csv")
X_test_orig = test_data.drop(['ID'], axis=1)
ids = test_data[['ID']].as_matrix()
ids = np.int32(ids)

In [7]:
X_test_preproc = pipe.transform(X_test_orig)
predictions_test = clf.predict_proba(X_test_preproc)

In [8]:
submission_1 = np.hstack((ids, predictions_test))
submission_1 = pd.DataFrame(data=submission_1[:,[0,2]],    # get only the values of the positive class
                            index=submission_1[:,0],    # 1st column as index too
                            columns=["ID","TARGET"], dtype=np.float32)
submission_1[["ID"]] = submission_1[["ID"]].astype(int)
submission_1.to_csv('submissions/santander-submission-4.(xgb-300 estimators,L2=3).txt',
                    index=False, dtype={'ID':np.int64, 'TARGET':np.float64})